# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

630

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
latitudes = []
longitudes = []
Max_temp = []
Humidities = []
Clouds = []
Wind_speed = []
Country = []
Date = []
Cities_New = []

    # Build query URL and request your results in Celsius
query_url = f"{url}appid={weather_api_key}&q={city}&units={units}"

for city in cities:
    try:
        weather_response = requests.get(f"{url}appid={weather_api_key}&q={city}&units={units}").json()
        latitudes.append(weather_response['coord']['lat'])
        longitudes.append(weather_response["coord"]["lon"])
        Max_temp.append(weather_response["main"]["temp_max"])
        Humidities.append(weather_response["main"]["humidity"])
        Clouds.append(weather_response["clouds"]["all"])
        Wind_speed.append(weather_response["wind"]["speed"])
        Country.append(weather_response["sys"]["country"])
        Date.append(weather_response["dt"])
        Cities_New.append(weather_response["name"])
        print(f"City {city} found...")
    except KeyError:
        print("City not found. Skipping...")
        
    
print("----------------Request Complete----------------")


City nanakuli found...
City praia found...
City santa marta found...
City not found. Skipping...
City upernavik found...
City alakurtti found...
City ushuaia found...
City hobyo found...
City yellowknife found...
City geraldton found...
City kerouane found...
City cabo san lucas found...
City albany found...
City kloulklubed found...
City sabang found...
City new norfolk found...
City not found. Skipping...
City biltine found...
City hilo found...
City chokurdakh found...
City vaini found...
City rikitea found...
City tuatapere found...
City shar found...
City port augusta found...
City thompson found...
City russkaya polyana found...
City sturlic found...
City butaritari found...
City seoul found...
City buala found...
City not found. Skipping...
City imuris found...
City punta arenas found...
City brae found...
City atuona found...
City esperance found...
City dzerzhinsk found...
City kavieng found...
City norman wells found...
City san pedro found...
City staryy merchyk found...
Cit

City warmbad found...
City talnakh found...
City oistins found...
City not found. Skipping...
City zajecar found...
City bundaberg found...
City kavaratti found...
City kodinsk found...
City not found. Skipping...
City nemuro found...
City ilulissat found...
City zapolyarnyy found...
City inhambane found...
City vitina found...
City asau found...
City mana found...
City roald found...
City walla walla found...
City not found. Skipping...
City not found. Skipping...
City olinda found...
City fairbanks found...
City sao joao da barra found...
City porto novo found...
City ribeira grande found...
City tefe found...
City sarishabari found...
City san carlos de bariloche found...
City not found. Skipping...
City salto found...
City not found. Skipping...
City korla found...
City bagdarin found...
City mahebourg found...
City hokitika found...
City fuxin found...
City phibun mangsahan found...
City not found. Skipping...
City beringovskiy found...
City chumikan found...
City longyearbyen fou

In [4]:
print(longitudes)
print(Cities_New)

[-158.1547, -23.5087, -74.199, -56.1549, 30.3491, -68.3, 48.5268, -114.3525, 114.6, -9.0167, -109.9124, -73.9662, 134.2556, 95.3214, 147.0587, 20.9167, -155.09, 147.9167, -175.2, -134.9692, 167.6833, 81.0453, 137.7667, -97.8558, 31.9201, 15.7781, 172.7902, 126.9778, 159.5921, -110.8667, -70.9167, -1.353, -139.0333, 121.9, 43.4631, 150.7967, -126.8329, 121.0167, 35.759, 63.044, 173.8833, 125.4092, -66.3821, 69.9, 26.4384, 115.3333, 152.9167, -42.0278, -60.1458, -158.0072, 100.3354, 120.3719, 30.7861, -6.589, -11.5258, -53.4616, -69.3632, -87.1, 54.0924, -79.2353, -97.5169, 6.2642, -65.0192, -16.3333, 8.8879, 26.891, -51.1922, -57.7667, 125.4869, 130.9889, -72.3287, -110.6974, 88.0718, 55.6167, -63.5724, 145.7785, -57.5575, -75.978, 141.9667, 21.4167, 39.8333, -151.75, -171, -120.4579, -9.4469, 42.7678, -24.4956, -64.7981, -92.1041, -40.2925, 164.0167, 150.85, -113.5841, -37.6368, -52.7314, -0.25, -159.775, 16.139, 57.6608, -50.0119, -73.414, 92.75, -17.1, -121.4238, 10.7833, -115.95, -7

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [31]:
Cities_data = [Cities_New,Clouds,Country,Date,Humidities,latitudes,longitudes,Max_temp,Wind_speed]

Cities_data = pd.DataFrame(
    {'City': Cities_New,
     'Cloudiness': Clouds,
     'Country': Country,
     'Date': Date,
     'Humidity': Humidities,
     'Latitude': latitudes,
     'Longitude': longitudes,
     'Max Temp': Max_temp,
     'Wind Speed': Wind_speed,
    })

Cities_data.to_csv ('City_Data.csv',index = True, header=True)
Cities_data.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Nānākuli,75,US,1619307452,45,21.3906,-158.1547,29.00,4.63
1,Praia,40,CV,1619307452,78,14.9215,-23.5087,21.00,6.69
2,Santa Marta,20,CO,1619307453,62,11.2408,-74.1990,31.00,6.17
3,Upernavik,10,GL,1619307453,85,72.7868,-56.1549,-3.04,2.90
4,Alakurtti,100,RU,1619307453,86,66.9672,30.3491,0.00,2.68


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [32]:
Humidity_100 = Cities_data.loc[Cities_data['Humidity']>=100]
Humidity_100

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
34,Dzerzhinsk,90,RU,1619307461,100,56.2389,43.4631,3.00,5.00
145,Clyde River,90,CA,1619307453,100,70.4692,-68.5914,-8.00,6.69
148,Boyuibe,90,BO,1619307488,100,-20.4167,-63.2833,19.00,5.40
193,Petropavlovsk-Kamchatskiy,90,RU,1619307499,100,53.0452,158.6483,2.00,4.00
230,Tricase,0,IT,1619307508,100,39.9304,18.3553,11.11,4.12
243,Misawa,75,JP,1619307511,100,40.6836,141.3597,8.89,3.60
248,Kiryat Gat,16,IL,1619307480,100,31.6100,34.7642,18.33,2.17
249,Husavik,0,IS,1619307513,100,66.0449,-17.3389,4.00,1.03
261,Surat Thani,20,TH,1619307516,100,9.1667,99.2500,26.67,1.03
291,Santa Maria,90,BR,1619307523,100,-29.6842,-53.8069,18.00,1.03


In [33]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = Cities_data.drop(Cities_data[Cities_data['Humidity'] > 99].index, inplace = False)
clean_city_data

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Nānākuli,75,US,1619307452,45,21.3906,-158.1547,29.00,4.63
1,Praia,40,CV,1619307452,78,14.9215,-23.5087,21.00,6.69
2,Santa Marta,20,CO,1619307453,62,11.2408,-74.1990,31.00,6.17
3,Upernavik,10,GL,1619307453,85,72.7868,-56.1549,-3.04,2.90
4,Alakurtti,100,RU,1619307453,86,66.9672,30.3491,0.00,2.68
...,...,...,...,...,...,...,...,...,...
574,Imeni Poliny Osipenko,41,RU,1619307599,29,52.4201,136.4853,3.72,6.40
575,Avilés,0,ES,1619307599,94,43.5547,-5.9248,16.67,2.06
576,Tacuarembó,100,UY,1619307600,93,-31.7333,-55.9833,16.58,4.38
577,Napier City,0,NZ,1619307600,45,-39.4833,176.9167,20.00,3.13


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression